In [96]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
pd.options.display.max_colwidth = 500
from xgboost import XGBClassifier

In [122]:
basedir = "C:/Users/Alexander/DeepAnyMatch/result_data/MADC_w2v_steps_walklength1_3grams_2019_07_07_15_11_45_607118/"

gs = pd.read_csv(basedir+"oaei_gold_standard5best.csv", encoding="UTF-8", sep="\t", header=None)
gs.columns = ['src_id','tgt_id','prediction']
embs = pd.read_csv(basedir+"stratified_embeddings.csv", encoding="UTF-8", sep=",")
embs = embs[[col for col in embs.columns if re.match('x\d+', col) is not None]+['label']]
embs.columns = ["src_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('x\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['src_id'], right_on=['label'])
embs.columns = ["tgt_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('src_\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['tgt_id'], right_on=['label'])
gs.head()

,src_id,tgt_id,prediction,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,tgt_91,tgt_92,tgt_93,tgt_94,tgt_95,tgt_96,tgt_97,tgt_98,tgt_99,label_y
0,http://dbkwik.webdatacommons.org/marvel/resource/cyclops,http://dbkwik.webdatacommons.org/dc/resource/cyclops,0,1.465558,-0.082710,-0.745632,-0.786758,0.350929,-0.478478,-0.179442,...,0.147990,0.501297,0.094320,0.541944,0.145733,-0.129454,0.269310,0.028970,0.135499,http://dbkwik.webdatacommons.org/dc/resource/cyclops
1,http://dbkwik.webdatacommons.org/marvel/resource/cyclops,http://dbkwik.webdatacommons.org/dc/resource/cyclopes,1,1.465558,-0.082710,-0.745632,-0.786758,0.350929,-0.478478,-0.179442,...,-0.328086,0.328326,-0.088920,0.335600,0.062955,0.132012,0.375087,-0.037113,-0.025080,http://dbkwik.webdatacommons.org/dc/resource/cyclopes
2,http://dbkwik.webdatacommons.org/marvel/resource/traveler,http://dbkwik.webdatacommons.org/dc/resource/traveler,0,-0.031406,0.184219,-0.881043,-1.067674,0.643378,-0.059030,-0.188607,...,-0.211132,0.286894,-0.007575,0.181855,0.010306,0.063304,0.283267,-0.060635,-0.261845,http://dbkwik.webdatacommons.org/dc/resource/traveler
3,"http://dbkwik.webdatacommons.org/marvel/resource/category:1942,_october","http://dbkwik.webdatacommons.org/dc/resource/category:1942,_october",0,0.015250,0.091292,-0.023617,-0.224610,-0.295551,0.535513,-0.041300,...,-0.210166,0.336384,0.312924,0.376009,0.104327,-0.100973,0.478241,0.323386,0.446639,"http://dbkwik.webdatacommons.org/dc/resource/category:1942,_october"
4,http://dbkwik.webdatacommons.org/marvel/resource/ultimates_vol_1_10,http://dbkwik.webdatacommons.org/dc/resource/intimates_vol_1_10,0,0.220097,0.148958,0.014681,-0.429959,0.042188,0.023552,0.051817,...,-0.078353,0.195945,0.507232,0.362378,0.146630,-0.261519,0.617776,0.019280,-0.085587,http://dbkwik.webdatacommons.org/dc/resource/intimates_vol_1_10


In [ ]:
oaei_gold_standard3 = pd.read_csv(basedir+"oaei_gold_standard2.csv_merged.csv",sep="\t",encoding="UTF-8")
oaei_gold_standard3 = oaei_gold_standard3[['src_id','tgt_id','label']]
embs = pd.read_csv(basedir+"stratified_embeddings.csv", encoding="UTF-8", sep=",")
embs = embs[[col for col in embs.columns if re.match('x\d+', col) is not None]+['label']]
embs.columns = ["src_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('x\d+', col) is not None]] + ['label']
oaei_gold_standard3 = oaei_gold_standard3.merge(embs, left_on=['src_id'], right_on=['label'])
embs.columns = ["tgt_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('src_\d+', col) is not None]] + ['label']
oaei_gold_standard3 = oaei_gold_standard3.merge(embs, left_on=['tgt_id'], right_on=['label'])

oaei_gold_standard3 = extend_features(oaei_gold_standard3)
oaei_gold_standard3['syntactic_diff'] = oaei_gold_standard3.apply(lambda row: jacc(row['src_id'], row['tgt_id']), axis=1)
oaei_gold_standard3['plus_diff'] = oaei_gold_standard3.apply(lambda row: lev(row['src_id'], row['tgt_id']), axis=1)
oaei_gold_standard3 = oaei_gold_standard3.drop(['label'], axis=1)
oaei_gold_standard3.rename({'label_x':'label'}, inplace=True, axis='columns')
oaei_gold_standard3 = oaei_gold_standard3.drop(['label_y'], axis=1)
oaei_gold_standard3.head()

In [123]:
origindir = "C:/Users/Alexander/rdata2graph/data/madc/"
labels1 = dict()
categories1 = dict()
with open(origindir+"graph_triples_ma.nt", encoding="UTF-8", mode="r") as f:
    for line in f:
        line = str(line).lower()
        if '<http://www.w3.org/2000/01/rdf-schema#label>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/2000/01/rdf-schema#label ")
            labels1[line[0]] = line[1]
        if '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/1999/02/22-rdf-syntax-ns#type ")
            if line[1] not in ['http://www.w3.org/2002/07/owl#class','http://www.w3.org/1999/02/22-rdf-syntax-ns#property']:
                categories1[line[0]] = 'resource'
            else:
                categories1[line[0]] = line[1]
labels2 = dict()
categories2 = dict()
with open(origindir+"graph_triples_dc.nt", encoding="UTF-8", mode="r") as f:
    for line in f:
        line = str(line).lower()
        if '<http://www.w3.org/2000/01/rdf-schema#label>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/2000/01/rdf-schema#label ")
            labels2[line[0]] = line[1]
        if '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' in line:
            line = line.replace("<","").replace(">","").replace(" .\n","").split(" http://www.w3.org/1999/02/22-rdf-syntax-ns#type ")
            if line[1] not in ['http://www.w3.org/2002/07/owl#class','http://www.w3.org/1999/02/22-rdf-syntax-ns#property']:
                categories2[line[0]] = 'resource'
            else:
                categories2[line[0]] = line[1]

In [124]:
gs.loc[:,'src_category'] = 'resource'
gs.loc[:,'tgt_category'] = 'resource'
for index, row in gs.iterrows():
    try:
        gs.loc[index, 'src_category'] = categories1[row['src_id']]
    except KeyError:
        pass
    try:
        gs.loc[index, 'tgt_category'] = categories2[row['tgt_id']]
    except KeyError:
        pass
gs = gs.loc[gs.src_category == gs.tgt_category]
len(gs)

11062

In [125]:
from sklearn.metrics.pairwise import *
def extend_features(df):
    src_pattern = "src_\d+"
    tgt_pattern = "tgt_\d+"
    src_dim = int(len([elem for elem in [re.match(src_pattern, elem) is not None for elem in df.columns.values.tolist()] if elem==True]))
    tgt_dim = int(len([elem for elem in [re.match(tgt_pattern, elem) is not None for elem in df.columns.values.tolist()] if elem==True]))


    def dotproduct(v1, v2):
        result = list()
        for i in range(len(v1)):
            result.append([np.dot(v1[i], v2[i])])
        return np.array(result)

    def length(v):
        return np.sqrt(dotproduct(v, v))

    def angle(v1, v2):
        return np.arctan(dotproduct(v1, v2) / (length(v1) * length(v2)))

    a = np.array(df[["src_" + str(i) for i in range(src_dim)]].values.tolist())
    b = np.array(df[["tgt_" + str(i) for i in range(tgt_dim)]].values.tolist())
    print(".")
    df['src_tgt_angle'] = paired_cosine_distances(a, b)
    print(".")
    src_origin = np.full((len(df), src_dim), 0.0000001)
    tgt_origin = np.full((len(df), tgt_dim), 0.0000001)
    df['src_angle_to_origin'] = paired_cosine_distances(tgt_origin,a)
    #print(".")
    df['tgt_angle_to_origin'] = paired_cosine_distances(src_origin,b)
    df['src_veclen'] = length(a)
    df['tgt_veclen'] = length(b)
    df['src_tgt_veclen'] = paired_euclidean_distances(a,b)#.diagonal()#length(a-b)
    df.head()
    
    df.fillna(0, inplace = True)
    return df


In [126]:
gs = extend_features(gs)
#oaei_gold_standard3 = extend_features(oaei_gold_standard3)
len(gs)

.
.


11062

In [127]:
memo = {}

def jacc(s,t, n=3):
    s = labels1[s]
    t = labels2[t]
    t = set([t[i:i+n] for i in range(len(t)-n+1)])
    s = set([s[i:i+n] for i in range(len(s)-n+1)])
    return 1-len([gram for gram in s if gram in t])/max(len(s), len(t))

def lev(s,t, n=3):
    s = labels1[s]
    t = labels2[t]
    return levenshtein(s,t)/max(len(s),len(t))
    
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1

    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])

    return res
gs['syntactic_diff'] = gs.apply(lambda row: jacc(row['src_id'], row['tgt_id']), axis=1)
gs['plus_diff'] = gs.apply(lambda row: lev(row['src_id'], row['tgt_id']), axis=1)

In [120]:
gs.loc[gs.src_id=='http://dbkwik.webdatacommons.org/marvel/property/creators']

,src_id,tgt_id,prediction,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,tgt_category,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,syntactic_diff,plus_diff,total_score
63,http://dbkwik.webdatacommons.org/marvel/property/creators,http://dbkwik.webdatacommons.org/dc/property/creators,0,0.12029,0.421354,0.028653,-0.078129,-0.440138,0.550072,0.131631,...,http://www.w3.org/1999/02/22-rdf-syntax-ns#property,0.729241,1.081051,0.984249,3.998457,6.446167,6.601759,0.000000,0.000000,0
66,http://dbkwik.webdatacommons.org/marvel/property/creators,http://dbkwik.webdatacommons.org/dc/property/creator,1,0.12029,0.421354,0.028653,-0.078129,-0.440138,0.550072,0.131631,...,http://www.w3.org/1999/02/22-rdf-syntax-ns#property,0.780873,1.081051,1.050720,3.998457,5.608273,6.132926,0.272727,0.076923,0


In [86]:
#from sklearn.ensemble import RandomForestClassifier 
#from sklearn.linear_model import LogisticRegression
##oaei_gold_standard3 = pd.read_csv(basedir+"oaei_gold_standard2.csv_merged.csv",sep="\t",encoding="UTF-8")
#
#cols = [col for col in oaei_gold_standard3.columns if col not in ['plus_diff','syntactic_diff','label','Unnamed: 0','src_id','tgt_id','src_category','tgt_category']]#['src_tgt_angle', 'src_tgt_veclen', 'plus_diff', 'syntactic_diff']
#X, y = oaei_gold_standard3[cols], oaei_gold_standard3.label
#clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.9,0:0.1}).fit(X, y)
##XGBClassifier().fit(X, y)
##LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.9,0:0.1}).fit(X, y)
#        #random_state=0, solver='lbfgs', multi_class='ovr', class_weight={1:0.1,0:0.9}).fit(X, y)
#
#X = gs[cols]
#matchings = gs.loc[clf.predict(X)==1]
#matchings.loc[:,'confidence'] = clf.predict_proba(matchings[cols])[:,1]
#len(matchings)

In [128]:
matchings = gs.loc[gs.plus_diff<0.1]
len(matchings)

5915

In [129]:
gs = matchings
len(matchings)

5915

In [113]:
len(set(gs.src_id))
from gensim.models import Doc2Vec, Word2Vec
model = Word2Vec.load(basedir+"w2v.model")
def get_training_material(nid):
            res = list()
            with open(basedir+"w2v_training_material.csv", mode="r", encoding="UTF-8") as f:
                for line in f:
                    if nodeid in line.split(" "):
                        for w in line.split(" "):
                            yield w

def mergedf(df1, df2):
            if df1 is None:
                return df2
            else:
                return df1.append(df2, ignore_index=True)


In [114]:
gs.loc[:,'total_score'] = 0

In [117]:
progress = 0
matchings = None
total = len(set(gs.src_id))
for nodeid in set(gs.src_id):#.union(gs.tgt_id)
                possible_matches_for_nodeid = gs.loc[(gs.src_id==nodeid) ]
        
                #possible_matches.loc[((possible_matches.src_id==nodeid) & (possible_matches.tgt_id.isin(get_possible_matches(nodeid))))]



                progress += 1
                if len(possible_matches_for_nodeid)<1:
                    continue
                elif len(possible_matches_for_nodeid) == 1:
                    matching_pair = possible_matches_for_nodeid.head(1)
                    matchings = mergedf(matchings, matching_pair)
                    continue
                
                
                #print(str(progress), end="\r")
                print("         Computing rank-features: " + str(int(100*progress/total)) + "%.", end='\r')
                # In[312]:



                #model.docvecs.most_similar(0)


                # In[313]:


                #print('Closest in general:')
                #for val in model.docvecs.most_similar(i):
                #    try:
                #        print(documents_ids_A[int(val[0])])
                #    except:
                #        try:
                #            print(documents_ids_B[int(val[0])])
                #        except:
                #            print(str(val[0]) + " not found")


                # In[314]:


                ##print('Closest in terms of cosine similarity:')
                ##vecs = model.docvecs.doctag_syn0[np.array(get_possible_matches(nodeid))]
                ##vecs = model.wv[get_possible_matches(nodeid)]
                ##x = cosine_similarity(np.array([model.wv[nodeid]]), vecs)
                ##x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                ##sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=False)
                ##sorted_x.loc[:,'cos_score'] = 0
                ctr = 1
                ##sorted_x.columns = ['cos_sim' if col==0 else col for col in sorted_x.columns]
                ##sorted_x.columns = ['cos_sim' if col==0 else col for col in sorted_x.columns]
                ##sorted_x['cos_sim'] = sorted_x['cos_sim'].astype('float64')
                sorted_x = possible_matches_for_nodeid.sort_values(by=['src_tgt_angle'], ascending=True)
                sorted_x.loc[:,'cos_score'] = 0
                #maximum = sorted_x.head(1).src_tgt_angle.values[0]
                #sorted_x.loc[:,'diff_to_max'] = 1.0 - sorted_x.loc[:, 'src_tgt_angle'] / maximum
                for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['cos_sim']))
                    sorted_x.loc[index, 'cos_score'] = row['cos_score'] + 1/ctr
                    ctr += 1
                #sorted_x = possible_matches_for_nodeid.sort_values(by=['confidence'], ascending=False)
                sorted_x.loc[:,'conficence'] = 0 
                sorted_x.loc[:,'confidence_score'] = 0 
                #ctr = 1
                #for index, row in sorted_x.iterrows():
                #    sorted_x.loc[index, 'confidence_score'] = row['confidence_score'] + 1/ctr
                #    ctr += 1

                # In[315]:


                ##print('Closest in terms of Euclidean distance:')print('Closest in terms of Euclidean distance:')
                #sorted_x2 = sorted_x
                ##vecs = model.wv[get_possible_matches(nodeid)]
                ##x = euclidean_distances(np.array([model.wv[nodeid]]), vecs)
                ##x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                ##sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
                sorted_x = sorted_x.sort_values(by=['src_tgt_veclen'], ascending=True)
                sorted_x.loc[:,'euclid_score'] = 0
                ctr = 1
                ##sorted_x.columns = ['euclid_sim' if col==0 else col for col in sorted_x.columns]
                for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['euclid_sim']))
                    sorted_x.loc[index, 'euclid_score'] = row['euclid_score'] + 1/ctr
                    ctr += 1



                #print('Closest in terms of syntax:')
                sorted_x2 = sorted_x
                #vecs = model.wv[get_possible_matches(nodeid)]
                def edits(v1, v2s):
                    res = list()
                    v1 = v1.split("/")[-1]
                    for v2 in v2s:
                        v2 = v2.split("/")[-1]
                        res.append(editdistance.eval(v1, v2)/min(len(v1), len(v2)))
                    return np.array([res])
                #x = edits(nodeid, get_possible_matches(nodeid))
                #x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
                #sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
                sorted_x = possible_matches_for_nodeid.sort_values(by=['syntactic_diff'], ascending=True)
                sorted_x.loc[:,'syntax_score'] = 0
                ctr = 1
                sorted_x.columns = ['syntax_diff' if col==0 else col for col in sorted_x.columns]
                #for index, row in sorted_x.iterrows():
                    #print(row[1] + " - " + str(row['syntax_diff']))
                #    sorted_x.loc[index, 'syntax_score'] = row['syntax_score'] + 1/ctr
                #    ctr += 1



                sorted_x3 = sorted_x
                #o1 = get_training_material(nodeid)
                #inverted_dict = dict()
                #for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
                #    inverted_dict[tgt_id] = [tgt_id]
                #    for word in get_training_material(tgt_id):
                #        if word in inverted_dict.keys():
                #            inverted_dict[word].append(tgt_id)
                #        else:
                #            inverted_dict[word] = [tgt_id]

                #tgt_scores = dict()
                #for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
                #    tgt_scores[tgt_id]=0

                #for tupl in model.predict_output_word(o1, topn=99999999999999):
                #    if tupl[0] in inverted_dict.keys():
                #        for tgt_id in inverted_dict[tupl[0]]:
                #            tgt_scores[tgt_id] = tgt_scores[tgt_id] + tupl[1]
                sorted_x.loc[:,'probability'] = 0
                sorted_x.loc[:,'probability_score'] = 0
                #ctr=1
                #for item in sorted(tgt_scores.items(), key = lambda kv:(kv[1], kv[0]), reverse=True):
                #    sorted_x.loc[sorted_x.tgt_id==item[0],'probability_score'] = 1/ctr
                #    sorted_x.loc[sorted_x.tgt_id==item[0],'probability'] = float(item[1])
                #    ctr += 1

                # In[316]:


                #print('Closest in sum:')
                #sorted_x[['probability_score','probability']].merge(..., left_index=True, right_index=True)
                x = sorted_x[['probability_score','probability']].merge(sorted_x3['syntax_score'].to_frame().merge(sorted_x2, left_index=True, right_index=True), left_index=True, right_index=True)
                x.loc[:,'total_score'] = x['syntax_score'] + x['euclid_score'] + x['probability_score'] + x['confidence_score'] + x['cos_score']
                sorted_x = x.sort_values(by=['total_score'], ascending=False)
                ##sorted_x.columns = ['tgt_id' if col==1 else col for col in sorted_x.columns]
                for index, row in sorted_x.iterrows():#sorted_x.loc[sorted_x.total_score == max(sorted_x.total_score.values),:].iterrows():
                    matching_pair = pd.DataFrame([sorted_x.loc[index]])
                    matching_pair.loc[:,'src_id'] = nodeid
                    #print(nodeid + "\t" + row[1] + "\t" + str(row['total_score']) + "\t" + str(row['cos_score']) + "\t" + str(row['euclid_score']))
                    matchings = mergedf(matchings, matching_pair)
                

c:\users\alexander\anaconda3\envs\py36\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [92]:
import os
def create_elem(src_id, tgt_id):
    elem = '<map>\n<Cell>\n<entity1 rdf:resource="'+src_id+'"/>\n'
    elem = elem + '<entity2 rdf:resource="'+tgt_id+'"/>\n<relation>=</relation>\n'
    elem = elem + '<measure rdf:datatype="xsd:float">1.0</measure>\n</Cell>\n</map>'
    return elem

matchings_filename ="MADC_ML.csv"
#married_matches = pd.read_csv("C:/Users/Alexander/rdata2graph/data/sap_hilti_data/oaei_gold_standard1best.csv", sep="\t", encoding="UTF-8")
#married_matches.columns=['src_id','tgt_id','label']
starttag = '<?xml version="1.0" encoding="utf-8"?>\n<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n<Alignment>\n  <xml>yes</xml>\n  <level>0</level>\n  <type>??</type>\n  <onto1>\n    <Ontology rdf:about="darkscape">\n      <location>http://darkscape.wikia.com</location>\n    </Ontology>\n  </onto1>\n  <onto2>\n    <Ontology rdf:about="oldschoolrunescape">\n      <location>http://oldschoolrunescape.wikia.com</location>\n    </Ontology>\n  </onto2>\n'
endtag = '</Alignment>\n</rdf:RDF>'
#os.mkdir(basedir + matchings_filename.replace(".csv",""))
with open(basedir + matchings_filename.replace(".csv","") + str(os.sep) + 'darkscape~oldschoolrunescape~results.xml', "w+", encoding="UTF-8") as f:
        f.write(starttag)
        for index, row in matchings.iterrows():
            f.write(create_elem(str(row.src_id).replace("&","&amp;"), str(row.tgt_id).replace("&","&amp;"))+"\n")
        f.write(endtag)



In [63]:
#matchings = matchings_saved
#matchings = gs

In [130]:
matchings_saved=matchings
matchings = matchings.sort_values(by=['syntactic_diff','syntactic_diff'], ascending=[True, True])
married_matchings = None
ctr = 0
while len(matchings) > 0:
                ctr += 1
                row = matchings.head(1)
                married_matchings = mergedf(married_matchings, pd.DataFrame(row))
                matchings = matchings.loc[~(matchings.src_id == row.src_id.values[0]) & ~(matchings.tgt_id == row.tgt_id.values[0])]
                print(str(len(matchings)) + " left     ", end="\r")

In [131]:
import os
def create_elem(src_id, tgt_id):
    elem = '<map>\n<Cell>\n<entity1 rdf:resource="'+src_id+'"/>\n'
    elem = elem + '<entity2 rdf:resource="'+tgt_id+'"/>\n<relation>=</relation>\n'
    elem = elem + '<measure rdf:datatype="xsd:float">1.0</measure>\n</Cell>\n</map>'
    return elem

matchings_filename ="MADC_MARRIED_Syntax_2.csv"
#married_matches = pd.read_csv(basedir + matchings_filename, sep="\t", encoding="UTF-8")
starttag = '<?xml version="1.0" encoding="utf-8"?>\n<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n<Alignment>\n  <xml>yes</xml>\n  <level>0</level>\n  <type>??</type>\n  <onto1>\n    <Ontology rdf:about="darkscape">\n      <location>http://darkscape.wikia.com</location>\n    </Ontology>\n  </onto1>\n  <onto2>\n    <Ontology rdf:about="oldschoolrunescape">\n      <location>http://oldschoolrunescape.wikia.com</location>\n    </Ontology>\n  </onto2>\n'
endtag = '</Alignment>\n</rdf:RDF>'
os.mkdir(basedir + matchings_filename.replace(".csv",""))
with open(basedir + matchings_filename.replace(".csv","") + str(os.sep) + 'darkscape~oldschoolrunescape~results.xml', "w+", encoding="UTF-8") as f:
        f.write(starttag)
        for index, row in married_matchings.iterrows():
            f.write(create_elem(str(row.src_id).replace("&","&amp;"), str(row.tgt_id).replace("&","&amp;"))+"\n")
        f.write(endtag)

